## Imports

In [6]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from pdpbox import pdp
from plotnine import *

ModuleNotFoundError: No module named 'pdpbox'

In [ ]:
import geopy.distance

In [5]:
#from fastai.imports import *
#from fastai.structured import *

import pandas as pd

#from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [ ]:
PATH = "data/BDS_day4/"

In [ ]:
!ls {PATH}

*Question*

estimate the Expected Delivery Time

In [ ]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
    
def plot_pdp(feat, clusters=None, feat_name=None):
    feat_name = feat_name or feat
    p = pdp.pdp_isolate(m, x, x.columns, feat)
    return pdp.pdp_plot(p, feat_name, plot_lines=True, 
                        cluster=clusters is not None, n_cluster_centers=clusters)
def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

In [4]:
import pandas as pd

df_raw = pd.read_excel('data_audi.xlsx', sheet_name='data', parse_dates=["DIMMFIR", "DIMMFIN", "DNAISSA"])

In [7]:
add_datepart(df_raw, 'DIMMFIR\')

SyntaxError: EOL while scanning string literal (<ipython-input-7-edf5ac1a0c22>, line 1)

In [ ]:
add_datepart(df_raw, 'DIMMFIN')

In [8]:
add_datepart(df_raw, 'DNAISSA')

NameError: name 'add_datepart' is not defined

In [ ]:
train_cats(df_raw)

In [ ]:
df, y, nas = proc_df(df_raw) #, max_n_cat=7)

In [ ]:
n_trn = 5000

In [ ]:
X_train, X_valid = split_vals(df, n_trn)

In [ ]:
selFld = 'B2BB2C'
print(df_raw[selFld].cat.categories)
print(df_raw[selFld].describe())

plt.show()
plt.title(selFld)
df[selFld].hist()

In [ ]:
selFld = 'DNAISSAYear'
print(df_raw[selFld].describe())

plt.show()
plt.title(selFld)
df[selFld].hist()

plt.show()
plt.title(selFld+' - without errors')
df[selFld][df[selFld]<2000].hist()

In [ ]:
selFld = 'DNAISSAYear'
selFld2 = 'B2BB2C'
value2 = 1

 
plt.show()
plt.title(selFld+' - for '+selFld2+' = '+str(value2))
df[selFld][df[selFld2]==value2].hist()

value2 = 2

plt.show()
plt.title(selFld+' - for '+selFld2+' = '+str(value2))
df[selFld][df[selFld2]==value2].hist()

In [ ]:
selFld = 'CSEXEXX'
print(df_raw[selFld].cat.categories)
print(df_raw[selFld].describe())

plt.show()
plt.title(selFld)
df[selFld].hist()

In [ ]:
selFld = 'CSEXEXX'
selFld2 = 'B2BB2C'
value2 = 1

plt.show()
plt.title(selFld+' - for '+selFld2+' = '+str(value2))
df[selFld][df[selFld2]==value2].hist()

value2 = 2

plt.show()
plt.title(selFld+' - for '+selFld2+' = '+str(value2))
df[selFld][df[selFld2]==value2].hist()

In [ ]:
# predictF = 'CPOSTAL'

# df_raw = pd.read_excel(f'{PATH}data_audi.xlsx', sheet_name='data', parse_dates=["DIMMFIR", "DIMMFIN", "DNAISSA"])

# df_raw[predictF] = np.log(df_raw[predictF])

# add_datepart(df_raw, 'DIMMFIR')
# add_datepart(df_raw, 'DIMMFIN')
# add_datepart(df_raw, 'DNAISSA')

# train_cats(df_raw)
# df, y, nas = proc_df(df_raw, predictF) #, max_n_cat=7)

# n_trn = 12000

# X_train, X_valid = split_vals(df, n_trn)
# y_train, y_valid = split_vals(y, n_trn)

# m = RandomForestRegressor(n_estimators=140, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
# m.fit(X_train, y_train)
# print_score(m)

In [ ]:
predictF = 'PRIX_DIFF_BASE_CAT'

df_raw = pd.read_excel('data_audi.xlsx', sheet_name='data', parse_dates=["DIMMFIR", "DIMMFIN", "DNAISSA"])

# df_raw[predictF] = np.log(df_raw[predictF])

df_raw.drop(['NCHASSI', 'ID', 'NCLIPRO', 'VPRICAT', 'VPRIBAS', 'VPRIFAC', 'PRIX_REDUCTION_INT', 'PRIX_REDUCTION'], axis=1, inplace=True)

add_datepart(df_raw, 'DIMMFIR')
add_datepart(df_raw, 'DIMMFIN')
add_datepart(df_raw, 'DNAISSA')

train_cats(df_raw)
df, y, nas = proc_df(df_raw, predictF) #, max_n_cat=7)

n_trn = 12000

X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

m = RandomForestRegressor(n_estimators=140, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
fi = rf_feat_importance(m, df); fi[:10]

In [ ]:
def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

In [ ]:
plot_fi(fi[:10])

In [ ]:
# predictF = 'PRIX_DIFF_BASE_CAT'

# df_raw = pd.read_excel(f'{PATH}data_audi.xlsx', sheet_name='data', parse_dates=["DIMMFIR", "DIMMFIN", "DNAISSA"])

# add_datepart(df_raw, 'DIMMFIR')
# add_datepart(df_raw, 'DIMMFIN')
# add_datepart(df_raw, 'DNAISSA')

# train_cats(df_raw)
# df, y, nas = proc_df(df_raw, predictF) #, max_n_cat=7)

# n_trn = 12000

# X_train, X_valid = split_vals(df, n_trn)
# y_train, y_valid = split_vals(y, n_trn)

In [ ]:
df.plot('CTITJUR','PRIX_REDUCTION', 'scatter', title='Discount in percentage by juridic type ! redundancy NL/FR', alpha=0.01, figsize=(10,8));

In [ ]:
df[(df['PRIX_REDUCTION']<0.6)&(df['PRIX_REDUCTION']>-0.1)].plot('TYPE_CLIENT','PRIX_REDUCTION', 'scatter', title='Discount vs Type_Client [LEA Part. Cond., LEA Soc. Locataire, LEA prop., Part. Cond., Part. prop., Soc. prop.]', alpha=0.01, figsize=(10,8));

In [ ]:
selFld = 'TYPE_CLIENT'
print(df_raw[selFld].cat.categories)
print(df_raw[selFld].describe())

plt.show()
plt.title(selFld)
df[selFld].hist()

In [ ]:
df.plot('CMODPRO','PRIX_REDUCTION_INT', 'scatter', title='Discount in percentage (rounded) vs CAR Model', alpha=0.01, figsize=(10,8));

In [ ]:
df[df['VPRIBAS']<60000].plot('CMODPRO','VPRIBAS', 'scatter', title='Catalog price, excluding options', alpha=0.5, figsize=(10,8));

In [ ]:
selFld = 'CMODPRO'
plt.show()
plt.title("Amount of cars by model")
df[selFld].hist(bins=20)

In [ ]:
selFld = 'VPRIBAS'
plt.show()
plt.title("Distribution of Price")
df[selFld].hist(bins=20)

In [ ]:
fldSel2 = 'CENERGY'
fldSel = 'CMODPRO'

x_all = get_sample(df[df[fldSel]>-150], 8500)
ggtitle("Energy by Car Mode")
ggplot(x_all, aes(fldSel, fldSel2))+stat_smooth(se=True, method='loess')

In [ ]:
df.plot('CMODPRO','QCARKWS', 'scatter', alpha=0.01, figsize=(10,8));

In [ ]:
fldSel2 = 'QCARKWS'
fldSel = 'CMODPRO'
x_all = get_sample(X_train[X_train[fldSel]>-150], 1500)
ggplot(x_all, aes(fldSel, fldSel2))+stat_smooth(se=True, method='loess')

In [ ]:
df.plot('CMODPRO','QCYLIND', 'scatter', alpha=0.01, figsize=(10,8));

In [ ]:
fldSel2 = 'QCYLIND'
fldSel = 'CMODPRO'
x_all = get_sample(X_train[X_train[fldSel2]>2000], 8500)
ggplot(x_all, aes(fldSel, fldSel2))+stat_smooth(se=True, method='loess')

In [ ]:
df2, y2, nas2 = proc_df(df_raw, 'VPRIBAS', max_n_cat=7)

fldSel2 = 'VPRIFAC'
fldSel = 'CMODPRO'
x_all = get_sample(df2[df2[fldSel]>-150], 1500)
ggplot(x_all, aes(fldSel, fldSel2))+stat_smooth(se=True, method='loess')